In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import hyperiax
from jax.random import PRNGKey, split
import jax
from jax import numpy as jnp
from hyperiax.models import UpdateModel
from hyperiax.models.functional import pass_up
from hyperiax.execution import RedBlackExecutor
from hyperiax.mcmc import ParameterStore, FixedParameter
import jax
from functools import partial

In [15]:
key = PRNGKey(0)

In [16]:
tree = hyperiax.tree.builders.symmetric_tree(3,2)
subkey, key = split(key)
noise_tree = hyperiax.tree.initializers.initialize_noise(tree, key, (2,))

In [17]:
class MCMCExample(UpdateModel):
    def update(self, parent_value,children_values,node_value, parameters):
        if not children_values:
            return {'noise': parent_value['noise']}, True
        if not parent_value:
            return {'noise': children_values['noise'].mean(0)}, True
        
        parent_noise = parent_value['noise']
        children_noise = children_values['noise'].mean(0)

        result = (1-parameters['lambd'])*parent_noise+parameters['lambd'] *children_noise

        return {'noise': result}, True
    

In [27]:
from hyperiax.mcmc.varianceParameter import VarianceParameter


params = ParameterStore({
    'lambd': FixedParameter(0.5),
    'alpha': VarianceParameter(2)
    # put more parameters here
})

In [28]:
model = MCMCExample()
exe = RedBlackExecutor(model)

In [29]:
params.propose(subkey).values()

{'lambd': 0.5, 'alpha': Array(2.0491912, dtype=float32)}

In [32]:
it = exe.get_iterator(noise_tree)

for node in it:
    key, subkey = jax.random.split(key)
    proposed = params.propose(subkey)

    accepted = exe.update(node, proposed.values())
    if accepted:
        params = proposed